In [ ]:
%matplotlib inline
from notebook_init import *

out_root = Path('out/figures/biggan_style')
makedirs(out_root, exist_ok=True)

model = get_model('BigGAN-512', 'husky', device)
rand = lambda : np.random.randint(np.iinfo(np.int32).max)

In [ ]:
classes = ['husky', 'church']
base_seeds = [665823877, 419650361]
style_seeds = [1922988331, 286873059, 1376693511, 1853453896]
print(base_seeds, style_seeds)

num_keep = [1, 4, 8] # switch latent after first, fourth, and eighth layer
layer_names = ['layer1', 'layer4', 'layer8']

model.truncation = 0.9

for class_idx, class_name in enumerate(classes):
    print(class_name, base_seeds[class_idx])
    
    model.set_output_class(class_name)
    
    for n_base in num_keep:
        strip = []
    
        # Base
        z0 = model.sample_latent(1, seed=base_seeds[class_idx])
        out = model.sample_np(z0)
        
        # Resample style
        plt.figure(figsize=(25,25))
        for img_idx, seed in enumerate(style_seeds):
            z1 = model.sample_latent(1, seed=seed)
            
            # Use style latent after 'n_base' layers
            n_style = model.get_max_latents() - n_base
            z = [z0] * n_base + [z1] * n_style
            
            img = model.sample_np(z)
            strip.append(img)
            
            # Save individually
            layer_name = f'layer{n_base}'
            img_name = out_root / f'style_resample_{class_name}_{layer_name}_{img_idx}.png'
            im = Image.fromarray((255*img).astype(np.uint8))
            im.save(img_name)
            
        # Show strip
        plt.imshow(np.hstack(strip))
        plt.axis('off')
        plt.show()